<div style="
    background: #f0f8ff; 
    color: #3777ac; 
    font-family: 'Arial', sans-serif; 
    padding: 30px; 
    border-radius: 15px; 
    text-align: center; 
    box-shadow: 0 6px 12px rgba(0, 0, 0, 0.4);
    margin-bottom: 30px;
">
    <h1 style="font-size: 2.5em;">🧠 LMSYS - AI Chat Evaluator: Enhancing Human-AI</h1>
</div>


<div style="
    background: #f0f8ff; 
    color: #3777ac; 
    font-family: 'Comic Sans MS', cursive, sans-serif; 
    padding: 20px; 
    border-radius: 10px; 
    text-align: center; 
    box-shadow: 0 4px 8px rgba(0, 0, 0, 0.3);
    margin-bottom: 20px;
">
    <h1>Welcome to my Notebook! 🙏</h1>
    <p> This notebook is dedicated to the <strong>Chatbot Arena Competition</strong>, where we explore and innovate in the realm of conversational AI.</p>
    <p><strong>Competition Overview:</strong> The goal is to develop a chatbot that excels in understanding and engaging users through natural and relevant conversations. You'll find data, techniques, and insights here to help you craft the best chatbot possible.</p>
    <p>Thank you for joining this journey. Dive in, explore, and let’s make some magic with chatbots!</p>
</div>



# 
<h1 style="
    background: #f0f8ff; 
    color: #3777ac; 
    font-family: 'Comic Sans MS', cursive, sans-serif; 
    padding: 15px; 
    border-radius: 10px; 
    text-align: center; 
    box-shadow: 0 4px 8px rgba(0, 0, 0, 0.2);
    border: 2px solid #3776ab;
">
 1. Setup the Environment
</h1>


In [ ]:
!pip install tensorflow pandas numpy scikit-learn


# 
<h1 style="
    background: #f0f8ff; 
    color: #3777ac; 
    font-family: 'Comic Sans MS', cursive, sans-serif; 
    padding: 15px; 
    border-radius: 10px; 
    text-align: center; 
    box-shadow: 0 4px 8px rgba(0, 0, 0, 0.2);
    border: 2px solid #3776ab;
">
2. Importing Libraries
</h1>


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, LSTM, Concatenate
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


# 
<h1 style="
    background: #f0f8ff; 
    color: #3777ac; 
    font-family: 'Comic Sans MS', cursive, sans-serif; 
    padding: 15px; 
    border-radius: 10px; 
    text-align: center; 
    box-shadow: 0 4px 8px rgba(0, 0, 0, 0.2);
    border: 2px solid #3776ab;
">
3. Loading.. Data
</h1>





In [ ]:
# Load the dataset
train_df = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/train.csv')
test_df = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')


# 
<h1 style="
    background: #f0f8ff; 
    color: #3777ac; 
    font-family: 'Comic Sans MS', cursive, sans-serif; 
    padding: 15px; 
    border-radius: 10px; 
    text-align: center; 
    box-shadow: 0 4px 8px rgba(0, 0, 0, 0.2);
    border: 2px solid #3776ab;
">
4. Preprocessing Text Data
</h1>








In [ ]:
# Concatenate responses and prompts for tokenization
texts = train_df['prompt'] + ' ' + train_df['response_a'] + ' ' + train_df['response_b']
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

# Convert text to sequences
def text_to_sequences(texts):
    sequences = tokenizer.texts_to_sequences(texts)
    return pad_sequences(sequences, maxlen=500)  # Adjust maxlen based on your needs

# Process training data
X_prompt_a = text_to_sequences(train_df['prompt'])
X_response_a = text_to_sequences(train_df['response_a'])
X_response_b = text_to_sequences(train_df['response_b'])

# Labels
y_a = train_df['winner_model_a']
y_b = train_df['winner_model_b']
y_tie = train_df['winner_tie']

# Split data for training
X_train_a, X_val_a, y_train_a, y_val_a = train_test_split(X_prompt_a, y_a, test_size=0.2, random_state=42)
X_train_b, X_val_b, y_train_b, y_val_b = train_test_split(X_prompt_a, y_b, test_size=0.2, random_state=42)
X_train_tie, X_val_tie, y_train_tie, y_val_tie = train_test_split(X_prompt_a, y_tie, test_size=0.2, random_state=42)


# 
<h1 style="
    background: #f0f8ff; 
    color: #3777ac; 
    font-family: 'Comic Sans MS', cursive, sans-serif; 
    padding: 15px; 
    border-radius: 10px; 
    text-align: center; 
    box-shadow: 0 4px 8px rgba(0, 0, 0, 0.2);
    border: 2px solid #3776ab;
">
5. Building the Model
</h1>

In [ ]:
# Define model parameters
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 100
max_len = 500

# Define inputs
input_prompt = Input(shape=(max_len,))
input_response_a = Input(shape=(max_len,))
input_response_b = Input(shape=(max_len,))

# Embedding layers
embedding_layer = Embedding(vocab_size, embedding_dim, input_length=max_len)

embedded_prompt = embedding_layer(input_prompt)
embedded_response_a = embedding_layer(input_response_a)
embedded_response_b = embedding_layer(input_response_b)

# LSTM layers
lstm_layer = LSTM(64)

encoded_prompt = lstm_layer(embedded_prompt)
encoded_response_a = lstm_layer(embedded_response_a)
encoded_response_b = lstm_layer(embedded_response_b)

# Concatenate encoded responses with prompt
concat_a = Concatenate()([encoded_prompt, encoded_response_a])
concat_b = Concatenate()([encoded_prompt, encoded_response_b])

# Dense layers for classification
dense_layer = Dense(64, activation='relu')

output_a = Dense(1, activation='sigmoid')(dense_layer(concat_a))
output_b = Dense(1, activation='sigmoid')(dense_layer(concat_b))
output_tie = Dense(1, activation='sigmoid')(dense_layer(concat_a))

# Model
model = Model(inputs=[input_prompt, input_response_a, input_response_b], 
              outputs=[output_a, output_b, output_tie])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()


# 
<h1 style="
    background: #f0f8ff; 
    color: #3777ac; 
    font-family: 'Comic Sans MS', cursive, sans-serif; 
    padding: 15px; 
    border-radius: 10px; 
    text-align: center; 
    box-shadow: 0 4px 8px rgba(0, 0, 0, 0.2);
    border: 2px solid #3776ab;
">
6. Training the Model
</h1>


In [ ]:
# Model Compilation
model.compile(
    optimizer='adam', 
    loss=['categorical_crossentropy', 'categorical_crossentropy', 'categorical_crossentropy'], 
    metrics=[['accuracy'], ['accuracy'], ['accuracy']]
)

# Model Training
history = model.fit(
    [X_train_a, X_train_b, X_train_b],
    [y_train_a, y_train_b, y_train_tie],
    validation_data=([X_val_a, X_val_b, X_val_b], [y_val_a, y_val_b, y_val_tie]),
    epochs=5,  # Adjust epochs as needed
    batch_size=32
)


# 
<h1 style="
    background: #f0f8ff; 
    color: #3777ac; 
    font-family: 'Comic Sans MS', cursive, sans-serif; 
    padding: 15px; 
    border-radius: 10px; 
    text-align: center; 
    box-shadow: 0 4px 8px rgba(0, 0, 0, 0.2);
    border: 2px solid #3776ab;
">
7. Evaluateing the Model
</h1>


In [ ]:
# Evaluate the model
eval_results = model.evaluate([X_val_a, X_val_b, X_val_b], [y_val_a, y_val_b, y_val_tie])

# Unpack the results
val_loss = eval_results[0]
loss_a = eval_results[1]
loss_b = eval_results[2]
loss_tie = eval_results[3]
acc_a = eval_results[4]
acc_b = eval_results[5]
acc_tie = eval_results[6]

print(f'Validation Loss: {val_loss}')
print(f'Loss for model_a predictions: {loss_a}')
print(f'Loss for model_b predictions: {loss_b}')
print(f'Loss for tie predictions: {loss_tie}')
print(f'Accuracy for model_a predictions: {acc_a}')
print(f'Accuracy for model_b predictions: {acc_b}')
print(f'Accuracy for tie predictions: {acc_tie}')


# 
<h1 style="
    background: #f0f8ff; 
    color: #3777ac; 
    font-family: 'Comic Sans MS', cursive, sans-serif; 
    padding: 15px; 
    border-radius: 10px; 
    text-align: center; 
    box-shadow: 0 4px 8px rgba(0, 0, 0, 0.2);
    border: 2px solid #3776ab;
">
8. Makeing Predictions
</h1>


In [ ]:
# Preprocess test data
X_test_prompt = text_to_sequences(test_df['prompt'])
X_test_response_a = text_to_sequences(test_df['response_a'])
X_test_response_b = text_to_sequences(test_df['response_b'])

# Predict
predictions = model.predict([X_test_prompt, X_test_response_a, X_test_response_b])

# Unpack predictions
predicted_a = predictions[0].flatten()
predicted_b = predictions[1].flatten()
predicted_tie = predictions[2].flatten()


# 
<h1 style="
    background: #f0f8ff; 
    color: #3777ac; 
    font-family: 'Comic Sans MS', cursive, sans-serif; 
    padding: 15px; 
    border-radius: 10px; 
    text-align: center; 
    box-shadow: 0 4px 8px rgba(0, 0, 0, 0.2);
    border: 2px solid #3776ab;
">
9. Prepareing the Submission File
</h1>


In [ ]:
# Format predictions for submission
submission = pd.DataFrame({
    'id': test_df['id'],
    'winner_model_a': predicted_a,
    'winner_model_b': predicted_b,
    'winner_tie': predicted_tie
})


# Save to CSV
submission.to_csv('/kaggle/working/submission.csv', index=False)


In [ ]:
submission.head()


<div style="
    background: #f0f8ff; 
    color: #3777ac; 
    font-family: 'Arial', sans-serif; 
    padding: 20px; 
    border-radius: 10px; 
    text-align: center; 
    box-shadow: 0 4px 8px rgba(0, 0, 0, 0.3);
    margin-top: 30px;
">
    <h2><strong>Thank You for Exploring My Notebook! 🙏<strong></h2>
    <p>I hope you found the content and insights valuable as we journeyed through the <strong>LMSYS - AI Chat Evaluator</strong> project together.</p>
    <p>Your feedback and support mean a lot to me. If you enjoyed this notebook and found it helpful, please consider giving it an <span style="background-color: #dde9f4; color: #3777ac; padding: 5px 10px; border-radius: 5px;"> <strong>upvote ⬆️</strong></span>. Your encouragement helps me continue improving and sharing useful content.</p>
    <p><span style="background-color: #dde9f4; color: #3777ac; padding: 5px 10px; border-radius: 5px;"><strong> Thank you once again for your time and support!</strong></span></p>
</div>
